## Variables à ajouter issues des recherches biblio

Variable(s): 

- Population des pays représentés 
- Variable indiquant si le pays est communiste ou pas 
- Variable indiquant si le pays est stable ou pas 
- Variable indiquant si le pays est émergeant ou pas ( otherwise pauvre , riche à peaufiner par la suite) 
- Variable indiquant si le pays a la culture du sport ou pas. 
- Variable indiquant le PIB par habitant 
- Variable indiquant les financements alloués aux sports (olympiques ou pas)


Il faut créer le train et le test avec des données 2024 , il faut les construire , il faut aussi savoir traiter le fait que cette fois ci la Russie ne sera pas disponible .
Faire indice : pays perturbé 1 avec Russie par exemple ou bien on enlève la Russie  (contexte géopolitique simple, on tah les années où j'ai eu un pb )

But c'est de prédire le nombre de médailles pour chaque pays , indicateur simple 

In [ ]:
# !pip install ydata_profiling

In [ ]:
import pandas as pd
import numpy as np
import ipython_memory_usage.ipython_memory_usage as imu
from describe_csv import *  # charge le fichier describe_csv.py par chunk de 10 000 (10**4)
from ydata_profiling import ProfileReport
import math
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import *
import scipy as sp
import scipy.stats
from pivottablejs import pivot_ui
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn import metrics
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split

from sklearn.pipeline import make_pipeline, make_union
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

imu.start_watching_memory()



# Introduction  (Business Understanding) 

## Détermination objectifs métier , analytique du projet 


Objectifs métiers du projet :

Promotion de la performance sportive : Utiliser les données des Jeux olympiques pour améliorer la performance sportive.

Optimisation des ressources : Identifier les sports les plus prometteurs pour optimiser les investissements sportifs.

Prédiction des tendances : Prédire les performances futures des pays et des athlètes en se basant sur les données historiques.

Analyse de la participation : Analyser la participation mondiale aux Jeux olympiques pour identifier les opportunités de croissance.

-------
Objectifs analytiques du projet :

Analyse des performances : Évaluer les performances des pays, des équipes et des athlètes aux Jeux olympiques.

Identification des facteurs de réussite : Identifier les clés du succès dans différents sports et disciplines.

Visualisation des données : Créer des visualisations interactives pour présenter les tendances et les comparaisons de manière compréhensible.

Modélisation prédictive : Développer des modèles prédictifs pour estimer les performances futures.

## Identifier les contraintes , limites et risque du projet

Qualité des données : Les données historiques peuvent être incomplètes ou imprécises.

Variabilité des performances : Les performances sportives sont influencées par de nombreux facteurs difficiles à quantifier.

Interprétation des résultats : Les résultats doivent être interprétés avec prudence en raison de la subjectivité des performances sportives.

Protection des données : Respecter les règles de confidentialité et de protection des données lors de l'analyse des données.

Dépendance aux technologies : Le projet dépend de technologies telles que l'analyse de données, la modélisation statistique, etc.

# 2. Data understanding (compréhension des données)


## 2.1.  Chargement des données

### Chargement des différentes bases de données + étude du chargement en mémoire

In [ ]:
df = pd.read_csv("data/athlete_events.csv")
df = df[(df['Year'] > 2000) & (df['Season'] == 'Summer')]

region_df = pd.read_csv("data/noc_regions.csv")
dictionary = pd.read_csv("data/dictionary.csv")

In [ ]:
# méthode pour calculer la taille réelle occupée par le dataframe : les index occupent aussi de la place en mémoire
df.memory_usage(index=True).sum()

32534052 octets = 32,53 MB =  32534,052 KB.

## 2.2 Décrire les données

### Charger les données avec describe_csv et indiquer les types optimaux pour chaque colonne. 

In [ ]:
describe_csv("data/athlete_events.csv", delimiter=',')

In [ ]:
describe_csv("data/noc_regions.csv", delimiter=',')
describe_csv("data/dictionary.csv", delimiter=',')

In [ ]:
print(df.isnull().sum())

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
# Find the unique values in year column.
df.Games.unique()

In [ ]:
# Find the unique values in city column.
df.City.unique()

### Utilisation pandas_profiling

In [ ]:
# profile = ProfileReport(df, title='Pandas Profiling Report')
# profile.to_file("report-profiling-test.html")

### EDA via pivottablejs

In [ ]:
# pivot_ui(df)

## EDA

### Target =  Medals : Nous allons construire des représentations graphiques nous permettant d'observer les Medals en fonction des autres variables.On effectue des transformations sur les données pour les besoins de cette analyse exploratoire (pas forcément les mêmes transformations que celles nécessaires lors de l'étape de Feature engineering).

#### Number of events in each sport over years [Heatmap]

In [ ]:
sport_year = df.pivot_table(index='Sport', columns='Year', values='Event', aggfunc=lambda x: len(x.unique()))
sport_year.fillna(0, inplace=True)
sport_year = sport_year.reindex(sport_year.sum(axis=1).sort_values(ascending=False).index)

# Affichage de la heatmap
plt.figure(figsize=(20, 20))
sns.heatmap(sport_year, annot=True, linewidths=0.05, cmap="RdYlGn")
plt.xlabel('Year', size=14, color="green")
plt.ylabel('Sport', size=14, color="green")
plt.title('Number of Events in Each Sport Over Years (Heatmap)', size=18, color="green")
plt.show()


#### Count plot for medals in summer olympics

In [ ]:
#Most Medals in Summer Olympics
data = df.groupby(['Event','Games'])['Medal'].value_counts()
sns.set(color_codes=True)                                               
sns.set_palette(sns.color_palette("muted"))
sns.countplot(y='Games', hue='Medal', data=df).set_title('Count plot for medals in summer olympics')

In [ ]:
df_sports = df.groupby(["Games","Sex"])["Sport"].count().reset_index().rename(columns={"Sport":"count"})
plt.figure(figsize=(25,10))
sns.barplot(x="Games", y = "count", hue = "Sex", data = df_sports, palette = "Blues" )

In [ ]:
# # RUSSIA
# df.loc[(df['Team'] == 'Soviet Union') & (df['NOC'] == 'URS'), ['Team', 'NOC']] = ['Russia', 'RUS'] # URSS période 1952-1988
df.loc[(df['Team'] == 'ROC') & (df['NOC'] == 'ROC'), ['Team', 'NOC']] = ['Germany', 'RUS'] # JO 2020

# # GERMANY
# df.loc[(df['Team'] == 'East Germany') & (df['NOC'] == 'GDR'), ['Team', 'NOC']] = ['Germany', 'GER']

# CHINA
df.loc[df['Team'] == "People's Republic of China", ['Team', 'NOC']] = ['China', 'CHN']

# UNITED STATES
df.loc[df['Team'] == "United States of America", ['Team', 'NOC']] = ['United States', 'USA']

#### Distribution of Age of Gold Medalists

In [ ]:
gold_medals = df[df['Medal'] == 'Gold']

plt.figure(figsize=(8, 6))  
plt.hist(gold_medals['Age'], bins=20, color='gold', edgecolor='black', alpha=0.7)  

plt.title('Distribution of Age of Gold Medalists', fontsize=16)  
plt.xlabel('Age', fontsize=14)  
plt.ylabel('Gold medal', fontsize=14)  
plt.xticks(rotation=45, fontsize=12)  


plt.show()
#


#### Evolution du nombres de médailles gagnés par les favoris

In [ ]:
import matplotlib.pyplot as plt

# Filter the data to include only gold medal winners
gold_medals = df[df['Medal'] == 'Gold']

# Group the data by year and count the number of gold medals won each year for each country
gold_medals_by_year_and_country = gold_medals.groupby(['Year', 'Team'])['Medal'].count().unstack()

# Select the top 10 countries with the most gold medals overall
top_10_countries = gold_medals.groupby('Team')['Medal'].count().nlargest(10).index

# Plotting the curve for the evolution of gold medals over time for the top 10 countries
plt.figure(figsize=(10, 6))  # Taille de la figure

for country in top_10_countries:
    plt.plot(gold_medals_by_year_and_country.index, gold_medals_by_year_and_country[country], 
             marker='o', markersize=6, linewidth=2, label=country)  # Courbe pour chaque pays avec la légende

plt.title('Evolution of Gold Medals Win Over Time (Top 10 Countries)', fontsize=16)  # Titre
plt.xlabel('Year', fontsize=14)  # Étiquette de l'axe x
plt.ylabel('Number of Gold Medals', fontsize=14)  # Étiquette de l'axe y
plt.xticks(fontsize=12)  # Taille de la police des étiquettes de l'axe x
plt.yticks(fontsize=12)  # Taille de la police des étiquettes de l'axe y

plt.grid(True)  # Afficher la grille
plt.legend(fontsize=10)  # Ajouter la légende
plt.tight_layout()  # Ajuster la mise en page
plt.show()


#### Evolution of Gold Medals per Athlete Over Time

In [ ]:
import matplotlib.pyplot as plt

# Calculate the total number of athletes per country
total_athletes_per_country = df.groupby('Team')['Name'].nunique()

# Filter the data to include only gold medal winners
gold_medals = df[df['Medal'] == 'Gold']

# Group the data by year and count the number of gold medals won each year for each country
gold_medals_by_year_and_country = gold_medals.groupby(['Year', 'Team'])['Medal'].count().unstack()

# Calculate the number of gold medals per athlete for each country and each year
gold_medals_per_athlete = gold_medals_by_year_and_country.div(total_athletes_per_country, axis=1)

# Select the top 10 countries with the most gold medals overall
top_10_countries = gold_medals.groupby('Team')['Medal'].count().nlargest(10).index

# Plotting the curve for the evolution of gold medals per athlete over time for the top 10 countries
plt.figure(figsize=(10, 6))  # Taille de la figure

for country in top_10_countries:
    plt.plot(gold_medals_per_athlete.index, gold_medals_per_athlete[country], 
             marker='o', markersize=6, linewidth=2, label=country)  # Courbe pour chaque pays avec la légende

plt.title('Evolution of Gold Medals per Athlete Over Time (Top 10 Countries)', fontsize=16)  # Titre
plt.xlabel('Year', fontsize=14)  # Étiquette de l'axe x
plt.ylabel('Number of Gold Medals per Athlete', fontsize=14)  # Étiquette de l'axe y
plt.xticks(fontsize=12)  # Taille de la police des étiquettes de l'axe x
plt.yticks(fontsize=12)  # Taille de la police des étiquettes de l'axe y

plt.grid(True)  # Afficher la grille
plt.legend(fontsize=10)  # Ajouter la légende
plt.tight_layout()  # Ajuster la mise en page
plt.show()


#### Total Number of Medals Won by Country and Medal Type

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Group the data by country and count the total number of medals won by each country and each type of medal
medals_by_country = df.groupby(['Team', 'Medal']).size().unstack(fill_value=0)

# Select the top 10 countries with the most total medals
top_10_countries = medals_by_country.sum(axis=1).nlargest(10).index

# Select the medal types
medal_types = ['Gold', 'Silver', 'Bronze']

# Set the position of each bar on the x-axis
x = np.arange(len(top_10_countries))

# Set the width of the bars
width = 0.2

# Assign colors to each medal type
colors = {'Gold': 'gold', 'Silver': 'silver', 'Bronze': 'peru'}

# Plotting the grouped bar graph
plt.figure(figsize=(12, 8))  # Taille de la figure

for i, medal in enumerate(medal_types):
    plt.bar(x - width + i * width, medals_by_country.loc[top_10_countries, medal], 
            width=width, label=medal, color=colors[medal])

plt.title('Total Number of Medals Won by Country and Medal Type', fontsize=16)  # Titre
plt.xlabel('Country', fontsize=14)  # Étiquette de l'axe x
plt.ylabel('Total Number of Medals', fontsize=14)  # Étiquette de l'axe y
plt.xticks(ticks=x, labels=top_10_countries, rotation=45, fontsize=12)  # Étiquettes de l'axe x
plt.yticks(fontsize=12)  # Taille de la police des étiquettes de l'axe y

# Supprimer les bordures de la figure
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

plt.legend(fontsize=12)  # Ajouter la légende
plt.tight_layout()  # Ajuster la mise en page
plt.show()


#### Strongest Evolution in Medals Won by Team

In [ ]:
# Filter the data to include only Summer Olympic Games between 2010 and 2020
summer_games = df[(df['Season'] == 'Summer') & (df['Year'] >= 2008) & (df['Year'] <= 2020)]

# Group the data by team and year, and calculate the total number of medals won by each team in each year
medals_by_year = summer_games.groupby(['Team', 'Year'])['Medal'].count().unstack()

# Calculate the difference in the number of medals won by each team between the first year and the last year
medal_difference = medals_by_year.iloc[:, -1] - medals_by_year.iloc[:, 0]

# Sort the teams based on the difference in the number of medals won in descending order
sorted_teams = medal_difference.sort_values(ascending=False)

# Select the top 5 teams with the largest increase in medals won
top_5_teams = sorted_teams.index[:10]

# Create a line chart showing the number of medals won by the top 5 teams in each year
plt.figure(figsize=(12, 6))
for team in top_5_teams:
    plt.plot(medals_by_year.loc[team], label=team)
plt.title('Strongest Evolution in Medals Won by Team', fontsize=18, color='#003f5c')
plt.xlabel('Year', fontsize=14, fontweight='bold', color='#003f5c')
plt.ylabel('Number of Medals', fontsize=14, fontweight='bold', color='#003f5c')
plt.xticks(fontsize=12, color='#003f5c')
plt.yticks(fontsize=12, color='#003f5c')
plt.grid(axis='y', linestyle='--', alpha=0.5, color='#003f5c')
plt.legend()
plt.tight_layout()
plt.show()


#### Number of Sports Over Time

In [ ]:
summer_sports_per_year = df[df['Season'] == 'Summer'].groupby('Year')['Sport'].nunique()

plt.figure(figsize=(12, 6))
plt.plot(summer_sports_per_year.index, summer_sports_per_year.values, color='#003f5c')
plt.title('Number of Sports Over Time (Summer)', fontsize=18, color='#003f5c')
plt.xlabel('Year', fontsize=14, fontweight='bold', color='#003f5c')
plt.ylabel('Number of Sports', fontsize=14, fontweight='bold', color='#003f5c')
plt.xticks(fontsize=12, color='#003f5c')
plt.yticks(fontsize=12, color='#003f5c')
plt.grid(axis='y', linestyle='--', alpha=0.5, color='#003f5c')
plt.tight_layout()
plt.show()

#### Last sport that apperead with most medals

In [ ]:
# Identifier les années où chaque sport est apparu pour la première fois
first_appearance = summer_games.groupby('Sport')['Year'].min()

# Sélectionner les 10 derniers sports apparus
latest_sports = first_appearance.sort_values(ascending=False).head(10)

# Filtrer les données pour inclure seulement les médailles des 10 derniers sports
latest_sports_medals = summer_games[summer_games['Sport'].isin(latest_sports.index)]

# Compter le nombre total de médailles par sport
medal_counts = latest_sports_medals.groupby('Sport')['Medal'].count().sort_values(ascending=False)

# Créer un graphique à barres
plt.figure(figsize=(10, 6))
bar_plot = medal_counts.plot(kind='bar', color='skyblue')

# Ajouter l'année d'apparition à chaque barre
for i, v in enumerate(medal_counts):
    sport_name = medal_counts.index[i]
    year = first_appearance[sport_name]
    bar_plot.text(i, v + 0.2, str(year), ha='center')

plt.title('Nombre total de médailles parmi les 10 derniers sports apparus')
plt.xlabel('Sport')
plt.ylabel('Nombre total de médailles')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

#### Sports with the Most Medals

In [ ]:
top_10_sports = df.groupby('Sport')['Medal'].count().nlargest(10)

fig, ax = plt.subplots(figsize=(12, 6))
ax.bar(top_10_sports.index, top_10_sports.values, color='#ffa600')
ax.set_title('Top 10 Sports with the Most Medals', color='#003f5c')
ax.set_xlabel('Sport', color='#003f5c')
ax.set_ylabel('Number of Medals', color='#003f5c')
ax.tick_params(axis='both', colors='#003f5c')
ax.grid(axis='y', linestyle='--', color='#003f5c')

for i, sport in enumerate(top_10_sports.index):
    ax.text(i, top_10_sports[sport] + 0.5, str(top_10_sports[sport]), color='black', ha='center')

plt.xticks(rotation=45, ha='right')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
bars = plt.barh(top_10_athletes['Athlete'] + ' - ' + top_10_athletes['Sport'], 
                top_10_athletes['Medal Count'], color='gold', edgecolor='black', alpha=0.7)

# Add the number of medals in the middle of each bar
for bar in bars:
    plt.text(bar.get_width(), bar.get_y() + bar.get_height()/2, f'{bar.get_width():.0f}', 
             va='center', ha='left', fontsize=10)

plt.xlabel('Medal Count', fontsize=14)
plt.ylabel('Athlete - Discipline', fontsize=14)  # Updated ylabel
plt.title('Top 10 Athletes with Most Medals', fontsize=16)
plt.gca().invert_yaxis()  # Invert y-axis to have the athlete with most medals on top
plt.grid(axis='x', linestyle='--', alpha=0.5)  # Add grid lines
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()


In [ ]:

# Filter the data to include only male and female athletes between 2000 and 2016
athletes_2000_2016 = df[(df['Year'] >= 2000) & (df['Year'] <= 2016)]

# Group the data by gender and year, and calculate the average height for each gender and year
average_height_by_gender_and_year_2000_2016 = athletes_2000_2016.groupby(['Sex', 'Year'])['Height'].mean()

# Create a line plot to visualize the variation of height over time between 2000 and 2016 for both male and female athletes
plt.figure(figsize=(10, 6))

# Plot the average height for male athletes
male_heights = average_height_by_gender_and_year_2000_2016['M']
plt.plot(male_heights.index, male_heights.values, color='blue', label='Male')

# Plot the average height for female athletes
female_heights = average_height_by_gender_and_year_2000_2016['F']
plt.plot(female_heights.index, female_heights.values, color='red', label='Female')

plt.title('Variation of Height for Male and Female Athletes over Time (2000-2016)', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Average Height (cm)', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True)
plt.legend()
plt.show()



# TEST PERSO (REVISIONS)

## Fusion des bases + Nettoyage

In [ ]:
df = pd.read_csv("data/athlete_events.csv")
df = df[(df['Year'] > 2000) & (df['Year'] <= 2016) & (df['Season'] == 'Summer')]

region_df = pd.read_csv("data/noc_regions.csv")
dictionary = pd.read_csv("data/dictionary.csv")

In [ ]:
df.shape

In [ ]:
# Fusion de la colonne 'Team' du DataFrame df avec la colonne 'Country' du DataFrame dictionary_df
merged_df = pd.merge(df, dictionary, left_on='Team', right_on='Country', how='left') # how = 'left' pour conserver toutes les lignes de df

# Fusion du DataFrame merged_df avec le DataFrame region_df en se basant sur la colonne 'NOC' dans merged_df et dans region_df
final_merged_df = pd.merge(merged_df, region_df, on='NOC', how='left')

# Affectation du résultat final à la variable df
df = final_merged_df
df.head()

df



In [ ]:
df.shape

## 3.2. Nettoyer les données

### 3.2.1. Données manquantes

In [ ]:
df.drop(columns=['Country', 'Code', 'region', 'notes'], inplace=True)

In [ ]:
# Vérification des valeurs NULL 
df.isnull().sum() 

#### Traitement des valeurs nulles


In [ ]:
df[df['Name'] == 'Ragnhild Margrethe Aamodt']

In [ ]:
# Calculer les moyennes de la taille et du poids par sexe
average_height = df.groupby('Sex')['Height'].mean()
average_weight = df.groupby('Sex')['Weight'].mean()

# Créer un DataFrame à partir des moyennes calculées
average_stats_df = pd.DataFrame({'Average Height': average_height, 'Average Weight': average_weight})
average_stats_df


In [ ]:
from sklearn.impute import SimpleImputer

df_male = df[df['Sex'] == 'M']
df_female = df[df['Sex'] == 'F']
columns_to_impute = ['Age', 'Height', 'Weight']

# Créer un imputer avec la stratégie 'mean' pour remplacer les valeurs nulles par la moyenne
imputer = SimpleImputer(strategy='mean')

df_male[columns_to_impute] = imputer.fit_transform(df_male[columns_to_impute])
df_female[columns_to_impute] = imputer.fit_transform(df_female[columns_to_impute])

df = pd.concat([df_male, df_female])


In [ ]:
df[df['Name'] == 'Ragnhild Margrethe Aamodt']

In [ ]:
df.isnull().sum()

In [ ]:
# Définir la fonction de mapping
def map_medal(medal):
    if medal == 'Gold':
        return 3
    elif medal == 'Silver':
        return 2
    elif medal == 'Bronze':
        return 1
    else:
        return 0

# Appliquer la fonction de mapping à la colonne 'Medal'
df['Medal'] = df['Medal'].map(map_medal)
df


Tracer, en un graphique, le boxplot des variables numériques : 

In [ ]:
df.boxplot()

In [ ]:
#Tracer le boxplot de la variable Age : 
df.boxplot(column='Age',return_type='axes')

In [ ]:
# On va modifier les plus hauts outliers identifiés pour Age par la valeur du 99ème quantile

cols = ['Age']
low = .05
high = .99 #Return values at the given quantile  0.99 (99% quantile)
quant_df =  df[cols].quantile([low, high])
print(quant_df)



In [ ]:
higher = quant_df.loc[high,'Age']
higher

def func(x):
    if x >= higher:
        return higher
    else:
        return x

df['Age'] = df['Age'].apply(func)

In [ ]:
#vérifier que pour les valeurs les plus hautes, on a bien modifié la valeur de fare par : 43 ans	

df[df['Age'].ge(quant_df.loc[high,'Age'])]

In [ ]:
df.boxplot(column='Age',return_type='axes')

In [ ]:
df.isnull().sum()

### 3.3.1 Indépendance des attributs numériques


In [ ]:
# # effectuer l'encodage du feature Medal (ajouter de colonnes au dataframe) : 

# encoder=preprocessing.OneHotEncoder(sparse_output=False)
# encoder.fit(df[['Medal']])
# columns_name = encoder.get_feature_names_out(['Medal'])
# df_encoded = pd.DataFrame(encoder.transform(df[['Medal']]))
# df_encoded.columns = columns_name
# df= pd.concat([df, df_encoded], axis=1)

In [ ]:
# effectuer l'encodage du feature Sex (ajouter de colonnes au dataframe) : 

# Effectuer l'encodage pour la variable 'Sex'
df_encoded = pd.get_dummies(df['Sex'], prefix='Sex')

# Concaténer le dataframe original avec le nouveau dataframe encodé
df = pd.concat([df, df_encoded], axis=1)

# Supprimer la colonne 'Sex' d'origine
df.drop('Sex', axis=1, inplace=True)
df



In [ ]:
pd.DataFrame(abs(df.corr(numeric_only=True)['Medal']).sort_values(ascending = False))


In [ ]:
#Les collonnes non numériques ne sont pas analysées
corrMatrix = df.corr(numeric_only=True)
sns.heatmap(corrMatrix, annot=True)
plt.show()

### 3.3.2 Indépendance des attributs catégoriels

Variables catégorielles non regardées encore : Team, Games, City



In [ ]:
# entre Team et City

ct_cat = pd.crosstab(df.Games, df.City)
ct_cat
chi2, p_value, _, _ = sp.stats.chi2_contingency(ct_cat)
print(chi2, p_value)

**Conclusion** : si p_value < 0.05 : Cabin et Embarked sont dépendants. 
**Il faudra supprimer un des deux par la suite.**

### 3.3.2. Feature engineering

#### Discrétisation des variables numériques continues

Nous allons discrétiser les 3 variables : 
* Age 
* Height 
* Weight 


https://www.kaggle.com/gunesevitan/titanic-advanced-feature-engineering-tutorial

In [ ]:
df.isnull().sum()

In [ ]:
# Discrétiser la variable 'Age' en 5 bins
# df['Age_Category'] = pd.cut(df['Age'], bins=5, labels=['Enfant', 'Adolescent', 'Jeune Adulte', 'Adulte', 'Personne âgée'])

# # Discrétiser la variable 'Height' en 5 bins
# df['Height_Category'] = pd.cut(df['Height'], bins=5, labels=['Très petit', 'Petit', 'Moyen', 'Grand', 'Très grand'])

# # Discrétiser la variable 'Weight' en 5 bins
# df['Weight_Category'] = pd.cut(df['Weight'], bins=5, labels=['Très léger', 'Léger', 'Moyen', 'Lourd', 'Très lourd'])

# Afficher les premières lignes du dataframe avec les nouvelles variables catégorielles


In [ ]:
df.dtypes

#### Création de nouveaux features

Nous allons créer 3 features : 

1. Organisateur : Pays organisateur : Créer une variable binaire qui indique si le pays de l'athlète est le pays hôte des Jeux Olympiques correspondants. Cela pourrait avoir un impact sur les performances des athlètes en raison du soutien local et des conditions environnementales favorables.


In [ ]:
# # Définir une fonction pour vérifier si la ville est le pays hôte
# def is_host_country(row):
#     if row['Team'] in row['Team']:
#         return 1
#     else:
#         return 0

# # Appliquer la fonction à chaque ligne du DataFrame
# df['Host_Country'] = df.apply(is_host_country, axis=1)
# df

### 3.3.3 Encodage des attributs discrets/catégoriels

Une étape à effectuer avant de pouvoir commencer notre modélisation est de convertir tous nos features catégoriels en nombres, car nos algorithmes ne peuvent prendre qu'un tableau de nombres en entrée, pas des noms ou des lettres. 
Choisissez les variables à tarnsformer, en utilisant des encoders adaptés. 

-----------------------------------------------------------------------------

LabelEncoder : Si vous avez des variables catégorielles ordinales avec un ordre spécifique, telles que 'Season' (saison des Jeux Olympiques), vous pouvez utiliser LabelEncoder pour mapper les catégories à des entiers.

OneHotEncoder : Pour les variables catégorielles nominales telles que 'Team' (nom de l'équipe), 'NOC' (code du Comité National Olympique), 'Sport' (nom du sport pratiqué) et 'City' (ville hôte des Jeux Olympiques), vous pouvez utiliser OneHotEncoder pour créer des variables binaires distinctes pour chaque catégorie.

DropColumns : Certaines variables catégorielles peuvent ne pas être pertinentes pour la modélisation, comme 'Name' (nom de l'individu) et 'Event' (nom de l'événement spécifique dans le sport). Vous pouvez envisager de supprimer ces colonnes.

In [ ]:
categorical_vars = df.select_dtypes(include='object').columns
print(categorical_vars)


In [ ]:
# from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# # Définir les colonnes à encoder
# ordinal_cols = ['Season']
# nominal_cols = ['Team', 'NOC', 'Sport', 'City']
# cols_to_drop = ['Name', 'Event']


# # Initialiser les encodeurs
# label_encoder = LabelEncoder()
# onehot_encoder = OneHotEncoder(sparse=False, drop='first')  # drop='first' pour éviter le piège de la variable fictive

# # Appliquer LabelEncoder aux variables ordinales
# df[ordinal_cols] = df[ordinal_cols].apply(label_encoder.fit_transform)

# # Appliquer OneHotEncoder aux variables nominales
# onehot_encoded = onehot_encoder.fit_transform(df[nominal_cols])
# onehot_df = pd.DataFrame(onehot_encoded, columns=onehot_encoder.get_feature_names_out(nominal_cols))

# # Supprimer les colonnes à supprimer
# df.drop(columns=cols_to_drop, inplace=True)

# # Concaténer les DataFrames encodés
# df = pd.concat([df, onehot_df], axis=1)

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Définir les colonnes à encoder
# ordinal_cols = ['Season']
# nominal_cols = ['Team', 'NOC', 'Sport', 'City']
cols_to_drop = ['Name', 'Event']


# Initialiser les encodeurs
label_encoder = LabelEncoder()
onehot_encoder = OneHotEncoder(sparse=False, drop='first')  # drop='first' pour éviter le piège de la variable fictive

# # Appliquer LabelEncoder aux variables ordinales
# df[ordinal_cols] = df[ordinal_cols].apply(label_encoder.fit_transform)

# # Appliquer OneHotEncoder aux variables nominales
# onehot_encoded = onehot_encoder.fit_transform(df[nominal_cols])
# onehot_df = pd.DataFrame(onehot_encoded, columns=onehot_encoder.get_feature_names_out(nominal_cols))

# Supprimer les colonnes à supprimer
df.drop(columns=cols_to_drop, inplace=True)

# Concaténer les DataFrames encodés
df = pd.concat([df, onehot_df], axis=1)


In [ ]:
df

In [ ]:
df.describe().T

In [ ]:
df.info()
df.columns.tolist()

In [ ]:
# X_TRAIN
feature_name = [
'Name',
 'Age',
 'Height',
 'Weight',
 'Team',
 'NOC',
 'Games',
 'Year',
 'Season',
 'City',
 'Sport',
 'Event',
 'Medal',
 'Population',
 'GDP per Capita',
 'Sex_F',
 'Sex_M']
# X_TRAIN
X = df[feature_name]


# Y_TRAIN --> target, variable à prédire
y = df['Medal']

# Entraînement du modèle (n_estimators : The number of trees in the forest.)
forest = RandomForestClassifier(n_estimators=100, n_jobs=-1) # n_jobs=-1 signifie autant de jobs que de coeurs de processeur  
forest.fit(X, y)